### Importing libraries

In [1]:
!pip install ipynb

In [2]:
import logging
from tensorflow.keras.callbacks import TensorBoard
from PIL import Image
import numpy as np
import time
import os
import sys
from ipynb.fs.defs.SiameseModel import Recognizer
import logging
import tensorflow as tf

### Image Dimensions

In [3]:
data_dimension = 128

### Input and target vectors

In [4]:
# Loading input and target vectors

X1 = np.load('x1.npy')
X2 = np.load('x2.npy')
Y = np.load('y.npy')

# Reshaping the target vectors
X1 = X1.reshape( ( X1.shape[0]  , data_dimension**2 * 3  ) ).astype( np.float32 )
X2 = X2.reshape( ( X2.shape[0]  , data_dimension**2 * 3  ) ).astype( np.float32 )

print( X1.shape )
print( X2.shape )
print( Y.shape )

(36, 49152)
(36, 49152)
(36,)


### Training and saving the model

In [5]:
# Creating an instance of siamese model
recognizer = Recognizer()

In [6]:
# Defining hyper parameters
parameters = {
    'batch_size' : 6 ,
    'epochs' : 5 ,
    'callbacks' : None , # [ TensorBoard( log_dir='logs/{}'.format( time.time() ) ) ] ,
    'val_data' : None
}

# Training the siamese model
recognizer.fit( [ X1 , X2 ], Y, hyperparameters=parameters)

6/6 [==============================] - 6s 1s/step - loss: 0.5905
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 49152)]      0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 49152)]      0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 64)           54243072    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)       

In [7]:
# saving the model
recognizer.save_model('model.h5')

### Loading images from directory

In [10]:
# Enter the path name untill 'images' [ Actual images are stored at ...images/sanyam/image0.png]
# Here enter the path till the local images
# Custom images : C:\Users\sanya\Desktop\Calgary\ENEL645\Project\images_old\custom_images\
# p1 : C:\Users\sanya\Desktop\Calgary\ENEL645\Project\images_old\p1\
# p2 : C:\Users\sanya\Desktop\Calgary\ENEL645\Project\images_old\p2\
dir_path_custom_images = input( 'Enter images directory path for custom images : ')
dir_path_class1_images = input( 'Enter images directory path for first class images : ')
dir_path_class2_images = input( 'Enter images directory path for second class images : ')

Enter images directory path for custom images : C:\Users\sanya\Desktop\Calgary\ENEL645\Project\images_old\custom_images\
Enter images directory path for first class images : C:\Users\sanya\Desktop\Calgary\ENEL645\Project\images_old\p1\
Enter images directory path for second class images : C:\Users\sanya\Desktop\Calgary\ENEL645\Project\images_old\p2\


In [18]:
# storing the numpy arrays of all images
custom_images = recognizer.prepare_images_from_dir( dir_path_custom_images  )
class_1_images = recognizer.prepare_images_from_dir( dir_path_class1_images )
class_2_images = recognizer.prepare_images_from_dir( dir_path_class2_images )

In [19]:
# printing shape of all the images
print("The shape of the numpy array of class first images", class_1_images.shape)
print("The shape of the numpy array of class second images", class_2_images.shape)
print("The shape of the numpy array of custom images", custom_images.shape)

The shape of the numpy array of class first images (3, 49152)
The shape of the numpy array of class second images (3, 49152)
The shape of the numpy array of custom images (5, 49152)


### Classifying images

In [22]:
# Predicting the score for each image

# scores stores a list of scores for each custom image with the train dataset
scores = list()

# labels is a list of labels for each custom image against the class label of trained image
labels = list()

# looping through all custom images
for image in custom_images:
    label = list()
    score = list()
    # looping across images of class 1
    for sample in class_1_images :
        image , sample = image.reshape( ( 1 , -1 ) ) , sample.reshape((1 , -1 ) )
        score.append( recognizer.predict( [ image , sample ])[0] )
        # appending label 0 to this class
        label.append( 0 )
    # looping across images of class 2
    for sample in class_2_images :
        image , sample = image.reshape( ( 1 , -1 ) ) , sample.reshape((1 , -1 ) )
        score.append( recognizer.predict( [ image , sample ])[0] )
        # appending label 1 for this class
        label.append( 1 )
    labels.append( label )
    scores.append( score )

### Analysing the scores and labels

In [23]:
print("The scores are: ", scores)
print("The labels are: ", labels)

scores = np.array( scores )
labels = np.array( labels )

print('The shape of the numpy array of labels are: ', scores.shape)
print('The shape of the numpy array of scores are:', labels.shape )
print('The scores are: ', scores)
print('The lables are: ', labels)

The scores are:  [[array([0.24661756], dtype=float32), array([0.05286223], dtype=float32), array([0.05286223], dtype=float32), array([0.09629604], dtype=float32), array([0.01379943], dtype=float32), array([0.11677787], dtype=float32)], [array([0.03369638], dtype=float32), array([0.0059104], dtype=float32), array([0.0059104], dtype=float32), array([0.5000699], dtype=float32), array([0.11610147], dtype=float32), array([0.55380374], dtype=float32)], [array([0.10076064], dtype=float32), array([0.28281587], dtype=float32), array([0.28281587], dtype=float32), array([0.06809771], dtype=float32), array([0.35752493], dtype=float32), array([0.05561629], dtype=float32)], [array([0.09994978], dtype=float32), array([0.28464115], dtype=float32), array([0.28464115], dtype=float32), array([0.06866992], dtype=float32), array([0.35959065], dtype=float32), array([0.05608997], dtype=float32)], [array([0.07179525], dtype=float32), array([0.08966142], dtype=float32), array([0.08966142], dtype=float32), arra

### Classifying into classes based on best score

In [24]:
for i in range( custom_images.shape[0] ) :
    index = np.argmax( scores[i] )
    label_ = labels[i][index]
    print( 'IMAGE {} is {} with confidence of {}'.format( i+1  , label_ , scores[i][index][0] ) )

IMAGE 1 is 0 with confidence of 0.24661755561828613
IMAGE 2 is 1 with confidence of 0.5538037419319153
IMAGE 3 is 1 with confidence of 0.35752493143081665
IMAGE 4 is 1 with confidence of 0.35959064960479736
IMAGE 5 is 1 with confidence of 0.5538042187690735
